# The goal is to scan the MSA to Phrogs
***

### With the truncated MSA

In [ ]:
import pandas as pd 
import os 
import subprocess
from multiprocessing import Pool

path_trunc = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/truncated_msa"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/phrogs_trunc_out"
path_phrogs="/home/conchae/databases/Phrogs/phrogs_hhsuite_db/phrogs"

paths_trunc = [f"{path_trunc}/{file}" for file in os.listdir(path_trunc)]

def scan_phrogs(path_query) :
    path_phrogs="/home/conchae/databases/Phrogs/phrogs_hhsuite_db/phrogs"
    index = path_query.split("/")[-1].split(".fasta")[0]
    path_index_out =  f"{path_out}/{index}"
    hhmscan_cmmd = f"hhsearch -i {path_query} -d {path_phrogs} -blasttab {path_index_out}.tab -o  {path_index_out}.hmm"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    scan_out, scan_err = hhmscan_process.communicate()
    with open(f"{path_files}/hhsearch_done","a+") as outfile :
        outfile.write(f"{path_query}\n")
        
if __name__ == '__main__':
    with Pool(30) as p:
        p.map(scan_phrogs, paths_trunc)

# **********************************************
#!/bin/bash
#BATCH --job-name=phrogs_PPT__
#SBATCH --qos=long
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=60 
#SBATCH --mem=120gb 
#SBATCH --time=5-00:00:00 
#SBATCH --output=phrogs_PPT__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/script_files/uniref.hmm.py

***
# Scan all the candidates proteins to Phrogs :



In [ ]:
import os 
import pandas as pd 
import subprocess
from multiprocessing.pool import ThreadPool

path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_phageboost = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_store = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/Phrogs_prediction_20042023"

desirable_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.final.desired_ipr.csv", sep="\t", header= 0)
seqid_df = desirable_df.drop_duplicates(subset = ["index_seq"] , keep = "first")

paths = []
for _,row in seqid_df.iterrows() :
    seqid = row["index_seq"]
    strain = row["protein_name"].split("__")[0]
    phage_id = row["protein_name"].split("__")[1]
    msa = f"{row['protein_name']}.MSA.a2m"
    path_file = f"{path_phageboost}/{strain}/mmseqs_out/{phage_id}/{msa}"
    a = (seqid , path_file)
    paths.append(a)

def scan_phrogs(tuple_path) :
    path_phrogs="/home/conchae/databases/Phrogs/phrogs_hhsuite_db/phrogs"
    index = tuple_path[0]
    path_query = tuple_path[1]
    path_index_out =  f"{path_store}/{index}"
    hhmscan_cmmd = f"hhsearch -i {path_query} -d {path_phrogs} -blasttab {path_index_out}.tab -o  {path_index_out}.hmm"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    scan_out, scan_err = hhmscan_process.communicate()
    print(path_query ,scan_out )
    with open(f"{path_db}/hhsearch_done.2004.txt","a+") as outfile :
        outfile.write(f"{path_query}\n")
        
if __name__ == '__main__':
    with ThreadPool(20) as p:
        p.map(scan_phrogs, paths)
        
# ******************************************************************        
#!/bin/bash
#BATCH --job-name=phrogs_PPT__
#SBATCH --qos=medium
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=40 
#SBATCH --mem=120gb 
#SBATCH --time=4-00:00:00 
#SBATCH --output=phrogs_PPT__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/script_files/part_III/phrogs_2004.py

> Scan the results 

In [ ]:
import os
import random
import sys
import pandas as pd
from Bio import SeqIO
from tqdm import tqdm
    # Remote
path_phrogs="/home/conchae/databases/Phrogs"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/phrogs_trunc_out"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_store = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/Phrogs_prediction_20042023"

results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.csv", sep="\t")
sequence_df = pd.read_csv(f"{path_db}/Results_III_sequences.csv", sep = "\t" ,names= ["index", "sequence"])

names=["#phrog","hostDomain","Avg_#AA","#ColMSA","#prot","Annotation","Category","RefSeq","Pfam_hit","GO_hit","KO_hit"]
blast_headers=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
phrogs_df = pd.read_csv(f"{path_phrogs}/PHROGS.csv", names=names, sep=",")

not_annotated = []
dico_annotation = {}
annotation = []

def get_annotation(tab_path) :
    blast_headers=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
    tab_out = pd.read_csv(f"{tab_path}", names=blast_headers, sep="\t")
    for index_hit, phrog_hit in tab_out.iterrows():
        if phrog_hit["evalue"] < 0.01 and phrog_hit["bitscore"] > 30 :
            annotation = phrogs_df[phrogs_df["#phrog"]==phrog_hit["sseqid"]]["Annotation"].values.tolist()[0]
            category = phrogs_df[phrogs_df["#phrog"]==phrog_hit["sseqid"]]["Category"].values.tolist()[0]
            stand_length = phrogs_df[phrogs_df["#phrog"]==phrog_hit["sseqid"]]["Avg_#AA"].values.tolist()[0]
            result = (annotation, category, phrog_hit["bitscore"])
            if category!= "unknown function":
                return result
                break
            if index_hit >=5 :
                break
        else :
            break
    else :
        return ("no significant annotation",)

paths = [f"{path_store}/{file}" for file in os.listdir(path_store) if file[-3:]=="tab"]

dico_results = {}
for path in tqdm(paths) :
    protein = path.split("/")[-1]
    index_annotation = get_annotation(path)
    dico_results[protein] = index_annotation

    
import pprint
pp = pprint.PrettyPrinter(width = 200)

pp.pprint(dico_results)   

indices = []
annotations = set()
for protein in dico_results :
    if dico_results[protein] != None :
        if dico_results[protein][0] in good_annotation :
            indices.append(protein.split(".tab")[0])
    else :
        indices.append(protein.split(".tab")[0])
            
        annotations.add(dico_results[protein][0])

pp.pprint(annotations)

good_annotation = ["EPS depolymerase",
"baseplate wedge subunit",
"colanic acid degradation",
"esterase/lipase",
"hydrolase",
"long tail fiber protein proximal connector",
"major tail protein",
"minor tail protein",
"tail fiber protein",
"tail fiber protein proximal subunit",
"tail protein",
"tail spike protein",
"tail spike protein with colonic acid degradation activity",
"virion structural protein"]

"levanase",

# ***********************
tail_fibers = []
endolysin = []
other = []

good_terms = ["baseplate hub","pectin lyase activity", "tail", "colanic acid degradation","virion structural protein", "structural protein with Ig domain" ,"EPS depolymerase"]
for __, prot in enumerate(annotation) :
    for good in good_terms :
        if prot[1].count(good) >0 :
            tail_fibers.append(prot)
            break
    else :
        if prot[1].count("endolysin") > 0 :
            endolysin.append(prot)
        else :
            other.append(prot)
        
other_annotation = set()
for prot in other :
    other_annotation.add(prot[1])
  

In [ ]:
good_seqindex = ['151', '973', '5194', '27', '2545', '88', '85', '3053', '3324', '6124', '3584', '4382', '4812', '4446', '1929', '1272', '1701', '1592', '5628', '1507', '1334', '3068', '1577', '4250', '1635', '3885', '3513', '2983', '5528', '5367', '3361', '6604', '880', '3831', '3857', '1025', '6677', '1369', '6704', '4358', '3966', '4077', '3203', '5457', '6167', '6757', '4746', '6951', '3988', '2578', '3648', '2382', '2799', '3737', '2306', '4652', '6091', '5015', '6759', '5675', '3538', '1757', '1893', '3647', '1523', '6823', '3095', '5298', '6349', '2933', '2870', '4465', '5237', '2321', '5521', '5913', '430', '5234', '528', '2193', '4173', '799', '3940', '5294', '2916', '1516', '4915', '1477', '3284', '2179', '6856', '2077', '5273', '6913', '1518', '7068', '1561', '2985', '1083', '3867', '87', '1170', '7135', '242', '6694', '3610', '3184', '7126', '1376', '4860', '6213', '2066', '5329', '5365', '1590', '1162', '4437', '5467', '2060', '3649', '1413', '6191', '2267', '3731', '2963', '2606', '6341', '2188', '2263', '2825', '173', '1927', '2694', '2276', '1371', '1702', '6038', '4175', '3519', '5511', '2946', '3247', '5964', '3298', '3536', '5362', '5380', '3370', '47', '257', '4981', '4934', '5112', '1954', '113', '6257', '5517', '215', '6970', '1969', '5343', '6762', '5608', '5314', '4215', '439', '2253', '3204', '5089', '764', '3234', '5388', '6062', '650', '120', '1151', '2041', '1051', '6366', '6900', '3848', '3939', '5060', '4354', '4618', '3253', '6941', '877', '1550', '2914', '410', '6017', '7062', '6346', '2816', '6044', '6405', '6024', '5261', '2768', '2124', '918', '2118', '4721', '1144', '232', '6009', '5128', '4699', '3466', '600', '5807', '765', '2265', '4964', '5893', '5187', '5869', '5579', '5890', '1552', '785', '4764', '2980', '3141', '6887', '5550', '4707', '1137', '5990', '4977', '2821', '3082', '3415', '6878', '5209', '499', '3619', '6672', '1725', '2336', '5629', '4859', '4638', '4343', '150', '5168', '92', '6688', '3999', '1305', '6441', '708', '2312', '4485', '4135', '1724', '5998', '887', '5707', '6085', '6949', '5777', '2392', '1610', '1481', '484', '3922', '3890', '2849', '4050', '2412', '1387', '6092', '1393', '5886', '4270', '6384', '4678', '458', '3352', '5481', '3354', '4306', '2520', '4029', '5575', '5075', '1365', '198', '6299', '4952', '6908', '3615', '7016', '3550', '4074', '1501', '2931', '2671', '5043', '3335', '292', '1375', '606', '4619', '3235', '3583', '1758', '6952', '7058', '1645', '6618', '2244', '4167', '3705', '579', '6181', '5412', '403', '2759', '2055', '4518', '2632', '2827', '5754', '2097', '3730', '1993', '3428', '972', '2996', '5610', '5712', '5123', '2959', '3502', '3345', '3381', '3157', '7137', '4837', '6490', '486', '2365', '871', '5472', '2973', '3365', '1513', '4026', '4675', '620', '2936', '5257', '4141', '7093', '2048', '6622', '881', '2764', '5795', '3088', '5596', '3958', '5678', '5636', '1168', '5369', '3001', '5510', '192', '966', '2682', '1094', '6061', '3898', '5711', '5164', '479', '6909', '5338', '5946', '4513', '75', '1110', '6211', '2389', '4491', '854', '2308', '310', '5823', '3785', '6967', '5450', '3949', '1382', '5907', '1768', '6699', '3337', '3680', '5944', '415', '2492', '6792', '3663', '3579', '454', '2842', '3321', '6785', '1065', '4302', '3251', '7032', '1599', '1670', '1985', '1583', '5490', '2663', '4108', '2861', '5157', '2551', '4836', '6162', '354', '6291', '903', '5080', '822', '1868', '1090', '1709', '6039', '5831', '2287', '5718', '3067', '3115', '3117', '1964', '4286', '6545', '3131', '861', '4689', '4261', '5111', '6389', '2734', '2725', '4599', '2779', '6522', '1496', '2402', '5563', '3598', '3312', '5443', '6889', '6617', '1953', '4162', '4551', '2710', '767', '1579', '4943', '4806', '6769', '5723', '2497', '1986', '1407', '1666', '2350', '5001', '303', '3016', '1805', '3937', '3666', '2626', '4230', '6075', '5796', '3170', '4038', '3728', '1378', '5444', '4196', '2236', '3231', '5980', '1923', '3553', '2643', '5140', '6067', '249', '3158', '2152', '2713', '4117', '352', '2411', '99', '6326', '3245', '1654', '5590', '1556', '5274', '4451', '1509', '3473', '5097', '833', '4509', '6712', '4858', '981', '796', '3299', '725', '5657', '6126', '5906', '1800', '4068', '6408', '137', '2476', '1300', '426', '686', '5867', '6509', '6270', '2075', '3870', '2215', '6789', '5469', '199', '2209', '6504', '4975', '3721', '4366', '3765', '6354', '1547', '1611', '6961', '3277', '3675', '2727', '2374', '5927', '4544', '1302', '3477', '6969', '2745', '322', '3799', '5116', '1794', '2941', '5689', '5466', '5559', '3515', '4039', '4072', '5587', '1056', '208', '2855', '2755', '2645', '2201', '4974', '2999', '2231', '4237', '3272', '3036', '789', '5633', '356', '1331', '7125', '614', '2888', '2574', '537', '5554', '5101', '1054', '3146', '1662', '79', '1195', '6904', '2575', '4488', '6816', '1875', '4145', '4756', '4998', '1514', '4190', '2099', '1682', '6579', '5687', '4588', '4519', '5456', '3209', '1395', '270', '5870', '367', '145', '6615', '4760', '5162', '5665', '3224', '1391', '6266', '1153', '5814', '1555', '1009', '5122', '2029', '420', '5243', '6417', '6303', '6086', '1233', '2320', '1931', '7039', '3081', '1745', '4555', '6125', '5646', '6046', '2087', '4527', '5029', '1396', '4033', '3596', '4477', '4076', '5601', '5047', '2903', '3216', '5837', '4170', '3945', '2563', '7010', '4772', '5638', '1576', '244', '5053', '2601', '651', '858', '3147', '4507', '1196', '1733', '3113', '797', '4336', '3178', '55', '6678', '4522', '3031', '1765', '3396', '1984', '2196', '265', '5816', '4712', '2525', '2526', '3391', '2681', '6751', '6601', '1669', '5463', '1876', '6510', '6881', '740', '5615', '2210', '3243', '852', '3118', '3367', '1437', '4432', '4042', '4362', '5947', '5817', '3421', '3414', '2833', '6436', '5422', '5147', '1617', '3985', '3738', '2895', '3061', '612', '104', '269', '4473', '1079', '3685', '1867', '3408', '2867', '6432', '2054', '1833', '6439', '5878', '2282', '1134', '4047', '2343', '5021', '6268', '3043', '94', '11', '6456', '485', '5882', '3683', '2429', '6468', '6219', '2229', '3755', '6552', '682', '4771', '3411', '697', '570', '3627', '3302', '4933', '4487', '2862', '5169', '2101', '138', '4125', '4380', '5622', '3419', '362', '3925', '4594', '5170', '599', '3971', '3162', '3979', '2103', '2322', '4183', '187', '6858', '4127', '6884', '2647', '272', '6867', '512', '135', '22', '5313', '5979', '6145', '1569', '2840', '4610', '5337', '3669', '4512', '297', '4472', '5855', '2301', '689', '4886', '1287', '1336', '5005', '5198', '4774', '3166', '2216', '5363', '1889', '4959', '13', '3191', '5020', '5894', '5745', '6894', '6852', '4715', '6736', '6173', '6014', '4498', '5098', '986', '1464', '2905', '1343', '3510', '5070', '5604', '4323', '6918', '3327', '2949', '676', '6933', '3863', '4255', '5040', '2954', '509', '2817', '7087', '5674', '4291', '6897', '6111', '5897', '5016', '1339', '4112', '2483', '6927', '4649', '3174', '4207', '4789', '6831', '4159', '1385', '2469', '2183', '990', '2025', '207', '7122', '820', '2672', '2524', '1517', '5211', '6551', '6895', '2620', '4613', '3009', '773', '1883', '3706', '3597', '1568', '1285', '1973', '5926', '428', '3632', '1206', '4768', '1441', '4600', '5375', '2471', '6745', '5956', '3333', '6306', '2740', '1823', '1797', '2104', '5216', '2953', '3808', '6997', '810', '836', '3994', '6069', '267', '3924', '1443', '7138', '5546', '1127', '6087', '4584', '5441', '4144', '7128', '4514', '2067', '835', '4154', '1920', '380', '1171', '3050', '5581', '2465', '2180', '1039', '2879', '1223', '6850', '4805', '6641', '4791', '342', '6869', '2443', '7130', '465', '3373', '6726', '6814', '2795', '341', '1266', '5406', '705', '1064', '2893', '4', '4095', '3546', '1933', '572', '1710', '864', '6964', '4529', '3357', '4480', '1063', '237', '3398', '3451', '2634', '6540', '5974', '2922', '3787', '3336', '6200', '5105', '1006', '6274', '667', '3789', '1434', '956', '5719', '816', '159', '472', '1629', '3955', '1145', '5178', '1741', '3512', '313', '3802', '6424', '5805', '5254', '936', '6412', '1219', '2133', '6011', '4611', '5414', '5664', '4511', '5393', '6265', '5544', '3232', '2992', '6430', '3442', '1720', '619', '2454', '2415', '6287', '519', '5799', '6029', '2579', '350', '3892', '631', '5804', '5177', '4900', '2293', '1582', '4100', '5413', '5930', '6228', '4884', '2602', '2670', '4221', '542', '3592', '3760', '2438', '6376', '6691', '2956', '3846', '6058', '4156', '6773', '1216', '2319', '2012', '4377', '3751', '3238', '947', '5182', '6217', '3116', '5585', '3374', '6612', '4637', '3850', '5949', '4515', '3580', '2250', '3965', '1638', '6102', '5484', '2482', '4316', '2093', '6795', '2804', '4814', '3143', '57', '4129', '4970', '2181', '6640', '5206', '6958', '4328', '5452', '1420', '167', '26', '2053', '5981', '3672', '5117', '2824', '5321', '6449', '1438', '3557', '6620', '1028', '4425', '3671', '4333', '425', '545', '1642', '3441', '5240', '4595', '3190', '6327', '2432', '2796', '5619', '1244', '6743', '6610', '3623', '2664', '2627', '4187', '4538', '625', '3123', '1750', '2018', '1844', '2958', '2198', '7070', '514', '4932', '2934', '1103', '2571', '7104', '6442', '1869', '5902', '2717', '2615', '5355', '3504', '1265', '6925', '4277', '4905', '6668', '5901', '3633', '4996', '3211', '2348', '2706', '5830', '6438', '3880', '2815', '4969', '2458', '851', '838', '3268', '1659', '5114', '2543', '4060', '4492', '1871', '3460', '4908', '2770', '4580', '911', '551', '1860', '6737', '2648', '5008', '6995', '6121', '2711', '408', '5051', '4438', '455', '2346', '6985', '2846', '5765', '1016', '4313', '2031', '6754', '6896', '4916', '2894', '935', '1840', '6244', '4421', '5327', '1831', '5092', '4923', '1526', '2635', '4182', '1068', '5801', '5641', '4830', '5086', '7123', '772', '6661', '3929', '5225', '5411', '1896', '3065', '6319', '1282', '247', '1947', '4420', '3878', '1537', '4713', '4280', '601', '1467', '4751', '3639', '1692', '116', '5293', '967', '5348', '5952', '5847', '4790', '4978', '5129', '3642', '6129', '2241', '3578', '5961', '1817', '6683', '4577', '3972', '2504', '999', '1242', '2052', '1436', '67', '4440', '1061', '771', '3380', '3832', '1174', '2979', '5548', '5062', '5773', '6513', '1428', '5512', '2314', '6277', '2932', '5082', '4885', '6829', '3963', '3692', '2609', '4927', '5955', '5790', '7084', '2616', '958', '5373', '4669', '366', '6782', '688', '1930', '548', '3308', '1812', '5663', '2513', '4389', '2843', '7080', '1033', '5677', '6307', '5667', '2249', '1539', '77', '5052', '6318', '5943', '4455', '6818', '4668', '3588', '2822', '4448', '5306', '988', '1298', '1093', '5290', '2559', '6295', '201', '5999', '4165', '5650', '5220', '5739', '5560', '568', '736', '4353', '3599', '248', '4385', '6182', '2889', '1938', '4598', '3733', '5004', '3744', '544', '3189', '264', '2421', '3111', '3723', '3911', '179', '3317', '2489', '846', '621', '3358', '3093', '2090', '3804', '6993', '4340', '4567', '4195', '1290', '1906', '101', '6387', '402', '6080', '7050', '3815', '5124', '1713', '6253', '879', '5310', '7100', '4201', '2010', '6947', '5071', '1968', '2790', '933', '6635', '4901', '2700', '757', '6874', '3175', '3793', '3916', '6563', '1046', '5772', '5883', '4189', '5449', '471', '6010', '6937', '3073', '870', '2880', '828', '4966', '4203', '316', '5120', '6176', '2006', '464', '533', '2864', '4899', '5109', '1246', '3686', '6506', '1288', '1045', '294', '221', '2122', '787', '3516', '4459', '5520', '2612', '4179', '1983', '1055', '5442', '6328', '1359', '4318', '526', '4976', '107', '6472', '6166', '3212', '4238', '1232', '1373', '2203', '4325', '6098', '730', '4917', '950', '843', '4018', '2377', '4398', '5197', '6280', '1887', '5501', '2002', '6627', '98', '1493', '3851', '7141', '3907', '193', '4733', '6990', '712', '2022', '5265', '6300', '2138', '3180', '5849', '1059', '2965', '5176', '1203', '1124', '3032', '1419', '1121', '2613', '3941', '5852', '3882', '3348', '246', '3287', '2315', '424', '1018', '6180', '5647', '4920', '760', '3982', '3696', '6070', '6464', '163', '4400', '3450', '1326', '1234', '3656', '6505', '5312', '4810', '6559', '4724', '3877', '2591', '5538', '6205', '5384', '394', '1714', '4431', '327', '5840', '7118', '2531', '5239', '6212', '3346', '1370', '2741', '236', '2070', '1104', '4055', '3416', '210', '3561', '1858', '448', '4786', '4010', '2503', '1922', '934', '2598', '1754', '5030', '2425', '4501', '5255', '2240', '4928', '3652', '2300', '6621', '4728', '3946', '2108', '6310', '3475', '2368', '5613', '3608', '2316', '6800', '4094', '1978', '6013', '3027', '3500', '2948', '3745', '5953', '1149', '376', '5006', '1448', '4800', '6738', '2784', '900', '6443', '2474', '4765', '4757', '3986', '1653', '1601', '5233', '4210', '4994', '755', '7006', '5866', '245', '6401', '899', '6671', '768', '409', '1427', '4897', '6544', '4547', '3489', '4384', '4815', '320', '4752', '416', '2744', '3954', '466', '6259', '6892', '6821', '2859', '1798', '3079', '7149', '3574', '4697', '7143', '582', '323', '3664', '2422', '784', '6160', '2952', '5549', '4105', '6857', '6281', '5639', '4694', '2810', '190', '445', '1313', '5531', '3773', '2001', '7015', '649', '3962', '3426', '2386', '4561', '2235', '5929', '5582', '1602', '991', '821', '6118', '4542', '4387', '747', '5301', '6977', '1243', '4941', '4434', '1197', '2177', '1175', '3961', '3206', '5670', '3476', '363', '5236', '2778', '3033', '6260', '6920', '3665', '5621', '5761', '3970', '2330', '5977', '4043', '6962', '6523', '1532', '6500', '2719', '4688', '4935', '4368', '3803', '6578', '6570', '786', '3171', '5012', '2051', '4737', '982', '1469', '3923', '1988', '6861', '4881', '6210', '5923', '10', '2568', '1966', '6484', '488', '6153', '6105', '3637', '4852', '3285', '357', '3563', '1117', '2639', '4331', '6915', '4629', '391', '6542', '6406', '5941', '2126', '1049', '1977', '818', '3511', '965', '5474', '638', '4150', '4548', '1082', '5065', '1827', '4508', '2829', '654', '5568', '6469', '2486', '256', '3703', '1853', '4308', '5465', '4777', '5656', '3024', '6454', '3039', '5445', '1796', '1604', '5648', '5076', '754', '2839', '1350', '3715', '857', '4304', '3102', '5300', '2260', '224', '4666', '626', '1842', '5819', '5002', '566', '5398', '447', '2026', '4219', '5453', '6285', '2803', '1697', '3474', '2499', '1324', '3622', '7048', '5049', '1558', '6903', '6602', '3595', '6966', '4867', '6429', '2750', '5527', '5671', '4025', '7081', '964', '1349', '4946', '6446', '3099', '2017', '3712', '180', '6149', '6059', '4467', '1943', '4320', '3125', '6169', '54', '3136', '3339', '3655', '2149', '4731', '211', '4360', '6963', '2459', '1575', '6702', '1024', '3568', '1453', '1361', '1836', '6146', '5296', '5738', '6202', '4381', '3144', '2059', '5207', '575', '2756', '2453', '2456', '41', '1900', '2495', '5357', '6186', '4356', '4160', '5757', '1072', '6569', '3899', '2156', '2169', '2629', '7145', '1386', '652', '2034', '2255', '1646', '629', '3062', '5447', '3300', '3390', '4314', '1845', '3562', '1293', '4676', '7037', '5356', '6880', '3547', '744', '6871', '1803', '2981', '3069', '1536', '2658', '6689', '4002', '6127', '6198', '278', '5099', '2159', '2274', '5630', '522', '2135', '1826', '4864', '44', '2337', '4352', '6971', '6084', '6777', '1830', '7021', '7054', '74', '1230', '1789', '6631', '3791', '1225', '7146', '2788', '5904', '3983', '4665', '4628', '3926', '5558', '559', '2783', '3269', '539', '1598', '4854', '2831', '4404', '1323', '1146', '2021', '5934', '7095', '5736', '2642', '4654', '5672', '6981', '5100', '2102', '3543', '4256', '4968', '157', '6725', '6055', '563', '4443', '770', '6870', '4209', '2536', '1624', '3040', '197', '1454', '5208', '1250', '5370', '1522', '4563', '2318', '560', '1684', '3456', '6807', '5666', '5916', '305', '254', '4627', '1161', '7134', '6278', '2401', '6344', '2580', '4680', '3818', '6695', '3693', '5856', '1228', '3575', '5072', '669', '883', '5025', '5960', '6574', '4957', '2212', '2039', '299', '5433', '583', '4007', '6379', '4671', '4907', '3405', '5519', '4803', '4253', '3756', '5364', '3548', '2325', '2960', '4725', '2618', '5395', '6463', '4910', '2036', '5041', '1048', '691', '1795', '2227', '2852', '948', '1211', '2974', '6178', '894', '2782', '4871', '6128', '943', '5419', '1078', '5077', '3868', '5288', '6890', '2433', '856', '2549', '16', '6827', '3387', '3467', '4120', '4269', '3045', '5323', '4322', '6157', '3198', '3167', '4597', '3894', '3372', '6787', '4524', '6729', '343', '5516', '5319', '1790', '6301', '4303', '315', '5505', '3019', '4137', '6930', '6411', '4225', '1899', '6216', '4447', '3447', '1699', '1696', '7019', '14', '6431', '5090', '3905', '6487', '2975', '920', '6410', '3257', '123', '3205', '3786', '989', '2395', '5885', '679', '5335', '1580', '2823', '6107', '1999', '5446', '737', '1312', '3267', '4616', '7106', '3781', '2257', '802', '1707', '866', '5438', '3007', '1253', '6073', '4971', '3624', '2488', '1600', '6005', '6081', '2151', '3091', '584', '6531', '6758', '6572', '3255', '1306', '1303', '6170', '6296', '5026', '6599', '801', '3085', '6320', '580', '4502', '1187', '3530', '388', '4847', '1910', '6226', '4021', '4281', '3792', '1058', '4004', '1766', '6592', '4126', '1317', '6511', '1779', '1329', '2080', '4254', '4263', '4767', '1490', '4388', '6739', '6068', '4893', '4819', '1426', '158', '592', '133', '706', '7147', '2107', '5863', '1685', '3621', '6015', '2771', '2557', '5752', '4192', '142', '2749', '3430', '3457', '4556', '125', '3797', '5387', '2753', '5148', '4222', '6236', '5250', '1498', '6740', '2431', '6189', '5345', '4802', '3893', '36', '2266', '3055', '2558', '5653', '5499', '684', '5093', '1344', '4006', '2237', '7004', '2089', '2760', '914', '2061', '1073', '2214', '4684', '5009', '5784', '590', '32', '5680', '3950', '2078', '2977', '4882', '6383', '1307', '1416', '6023', '556', '6932', '5034', '5525', '4495', '4778', '863', '196', '5232', '743', '1618', '6414', '3551', '6407', '718', '109', '2773', '1140', '6778', '4136', '3015', '1417', '1043', '506', '3990', '5782', '3084', '6888', '2270', '3425', '6022', '824', '6482', '2202', '5737', '6957', '6137', '3092', '5011', '2588', '1220', '4347', '7124', '1991', '674', '748', '2112', '1183', '414', '34', '4550', '5151', '6395', '2986', '3528', '3252', '4785', '2427', '1791', '1680', '7089', '4674', '1169', '6723', '5272', '1022', '1641', '2277', '1567', '6697', '5377', '3618', '1863', '5722', '6928', '5359', '1657', '2095', '4574', '4980', '1839', '2406', '5425', '4229', '1775', '6910', '604', '7113', '2167', '1135', '2891', '5567', '1980', '2746', '1743', '840', '6549', '1761', '6109', '3484', '6842', '1439', '4820', '2153', '1718', '5287', '806', '6122', '3402', '6539', '168', '5744', '2789', '2261', '898', '5013', '703', '3017', '1150', '4460', '5183', '6453', '2329', '2720', '50', '1248', '2445', '3223', '3920', '2619', '7017', '1625', '3933', '1008', '3070', '5821', '4390', '1160', '2176', '4530', '2776', '5311', '6690', '3991', '6391', '491', '1483', '5217', '808', '4372', '1017', '1524', '1813', '6363', '882', '3325', '2837', '5733', '5561', '4818', '2669', '4982', '4244', '992', '2765', '4456', '3042', '6516', '2577', '59', '701', '3620', '2171', '3054', '434', '5865', '3800', '2595', '3674', '6031', '5188', '3295', '1397', '2399', '1647', '7063', '3280', '524', '6263', '5248', '4878', '1316', '4596', '5258', '6478', '4143', '775', '6507', '3003', '5684', '3930', '4779', '6229', '4692', '6613', '5135', '5995', '719', '2668', '4604', '4063', '3845', '321', '6772', '4648', '1295', '2567', '4609', '5184', '4083', '6547', '6185', '3200', '1676', '665', '6580', '1279', '1035', '1364', '5573', '7038', '4543', '1163', '2224', '5972', '4851', '1182', '2323', '7056', '1294', '1471', '867', '672', '2978', '181', '4315', '3533', '5996', '494', '1698', '4054', '5562', '6644', '5526', '230', '3766', '5403', '7103', '2269', '6227', '2772', '2766', '148', '3828', '6155', '2830', '347', '5931', '4633', '5716', '844', '3397', '271', '2708', '2345', '42', '5503', '2603', '7101', '6514', '5378', '1452', '6741', '6404', '4705', '4011', '809', '3076', '64', '1136', '5658', '1108', '2538', '2251', '657', '4685', '422', '241', '6779', '377', '6954', '6611', '6931', '5631', '2166', '5668', '3271', '6543', '4435', '3565', '2192', '6607', '152', '5058', '5279', '5833', '3518', '6345', '3570', '1185', '3539', '5789', '139', '495', '3392', '5791', '2254', '6643', '5969', '3029', '2811', '4001', '6393', '5699', '1862', '1304', '4950', '2405', '720', '2514', '4321', '2011', '690', '1932', '3977', '2500', '2484', '1191', '727', '3628', '1815', '640', '1015', '4121', '4194', '4679', '4408', '1097', '7007', '3659', '7085', '3152', '2512', '4200', '4080', '3603', '939', '2738', '1763', '4853', '4065', '6372', '4342', '5033', '3282', '2553', '6594', '2333', '5690', '3119', '1996', '3344', '1254', '3389', '6768', '3465', '1631', '5813', '6797', '4540', '4486', '3710', '5246', '1689', '2418', '839', '5603', '5720', '6798', '1459', '4016', '4997', '6587', '6575', '5130', '3020', '6555', '6370', '2923', '5426', '4395', '1189', '3735', '6844', '2737', '677', '2806', '2296', '451', '5106', '5282', '1128', '4359', '5489', '2962', '3794', '330', '1348', '2649', '5868', '2388', '2007', '2995', '3349', '2143', '4118', '5471', '3182', '4993', '2351', '3427', '6333', '4582', '6771', '4363', '1280', '2381', '1963', '6732', '1573', '6755', '3748', '1497', '4414', '4670', '5918', '6006', '3617', '5390', '819', '5067', '3445', '7076', '4247', '222', '5083', '5832', '6978', '5900', '6595', '1549', '1510', '2313', '6462', '976', '5309', '5201', '6519', '2279', '6052', '1158', '3594', '782', '1389', '4855', '711', '4330', '6936', '1822', '5152', '4373', '4845', '5292', '6141', '5196', '1656', '1070', '170', '2020', '6095', '2661', '5454', '4327', '5191', '1762', '2199', '6955', '6323', '4835', '6822', '6418', '1649', '2464', '4075', '6825', '4798', '19', '3747', '5676', '4863', '5753', '2757', '6423', '541', '4710', '2449', '7040', '6882', '3762', '4834', '348', '1480', '4110', '5798', '6019', '2560', '4272', '618', '3226', '432', '1660', '5326', '1890', '4569', '3108', '6646', '901', '923', '2363', '6582', '5616', '6255', '1458', '2812', '331', '5205', '3852', '1487', '4397', '3248', '660', '3644', '6714', '1848', '1864', '286', '501', '5415', '3813', '2968', '1939', '4152', '6938', '4468', '6314', '5606', '3590', '6916', '5035', '1828', '2572', '3403', '4301', '1320', '565', '4579', '5751', '6923', '3288', '3713', '668', '714', '3246', '2906', '2252', '2434', '3376', '2562', '1574', '431', '2547', '5717', '1630', '6119', '4444', '2716', '3470', '5578', '6912', '3320', '308', '3264', '1909', '7120', '2347', '6791', '2147', '332', '4178', '5269', '5342', '637', '326', '1325', '3028', '4022', '6705', '6826', '4960', '7142', '1098', '1013', '2290', '2', '3388', '1958', '1351', '6479', '4464', '1769', '1308', '5459', '659', '2501', '266', '2544', '2801', '5515', '3777', '1607', '4583', '2355', '4163', '4662', '2291', '6375', '1105', '5371', '3161', '5069', '2774', '2408', '998', '1512', '4483', '2481', '6980', '884', '7090', '2341', '2887', '6352', '4300', '2582', '6780', '3080', '6680', '4739', '4013', '5295', '3124', '2797', '442', '547', '6841', '1004', '397', '6215', '644', '5421', '5079', '6972', '3229', '557', '2555', '505', '1214', '3640', '1447', '6177', '1346', '2697', '6989', '1820', '4379', '1781', '925', '1301', '4557', '51', '3237', '4822', '1278', '6361', '3919', '4062', '7152', '842', '6289', '6147', '4532', '6746', '1310', '2683', '926', '758', '6561', '6002', '4199', '4525', '3121', '4471', '4918', '1759', '2599', '4989', '2793', '3840', '6944', '2519', '7033', '2364', '5617', '4703', '7', '4745', '128', '1596', '162', '2899', '2625', '155', '1777', '258', '1504', '91', '2410', '1982', '1772', '6322', '3127', '3948', '6148', '4113', '4057', '1655', '945', '5872', '5806', '5302', '3952', '4990', '1627', '681', '7053', '7018', '4874', '5766', '4274', '1147', '5031', '1096', '3254', '427', '5635', '1546', '4541', '7059', '3495', '6373', '3826', '6774', '115', '6133', '6290', '4413', '6847', '1101', '4716', '1623', '6399', '0', '1671', '1422', '2600', '2417', '1694', '1435', '587', '6973', '2023', '1408', '6515', '597', '6261', '93', '1793', '4795', '2688', '6123', '261', '89', '4284', '5655', '598', '3889', '306', '3602', '2219', '229', '3469', '4299', '5127', '5536', '4391', '1619', '1322', '3444', '6806', '5915', '345', '1188', '5392', '5571', '2970', '6629', '3750', '1965', '65', '2441', '1665', '6901', '4227', '6037', '813', '2592', '7041', '400', '4614', '5214', '4734', '3998', '5014', '5289', '530', '6161', '4682', '5989', '1935', '2352', '1715', '4339', '2502', '5400', '4040', '4015', '3829', '3682', '7064', '538', '2739', '4061', '7028', '6360', '2610', '3056', '1810', '6877', '194', '3667', '4312', '1037', '2835', '2264', '2913', '3012', '285', '6940', '2000', '5483', '3491', '239', '4355', '2581', '6135', '3148', '106', '3928', '3914', '6815', '578', '2876', '3459', '3942', '4770', '5522', '3266', '5637', '3490', '3576', '550', '5827', '3503', '5125', '4504', '3154', '2633', '2813', '2548', '4894', '4341', '511', '692', '534', '3310', '4394', '4763', '483', '959', '4496', '4081', '3187', '984', '4949', '5502', '1970', '3523', '2164', '574', '3301', '1705', '2780', '4326', '3311', '6258', '892', '2131', '3463', '946', '5045', '3758', '1430', '334', '2259', '2675', '937', '3860', '1360', '3485', '5518', '3213', '1677', '4925', '4378', '1327', '1262', '3635', '7116', '3651', '2394', '475', '4672', '4660', '1208', '3296', '4101', '5331', '84', '5480', '3641', '1152', '4718', '2777', '1080', '628', '2509', '4132', '3871', '769', '5036', '1442', '1730', '6131', '5281', '6000', '2072', '1374', '3819', '2457', '2730', '6154', '4832', '5219', '1492', '4234', '6566', '5623', '2586', '4856', '68', '6999', '5598', '1691', '5891', '3135', '5235', '146', '5498', '5818', '608', '969', '437', '3865', '1717', '1433', '7150', '4427', '5437', '2446', '3274', '2521', '2479', '1173', '6649', '1613', '1726', '4909', '761', '6786', '7001', '2455', '1109', '4591', '3436', '4265', '1041', '6364', '169', '5721', '4003', '6911', '3604', '930', '118', '5241', '6082', '2145', '6079', '527', '1892', '4549', '5408', '5694', '3957', '273', '1563', '5166', '1719', '6976', '1664', '1380', '790', '2358', '741', '5593', '3023', '4759', '6247', '1192', '759', '811', '2360', '1734', '2057', '4411', '5003', '1675', '4067', '2989', '2976', '2384', '6984', '1859', '5843', '7042', '421', '6294', '4224', '5154', '460', '1089', '5226', '2523', '3541', '1440', '2044', '3844', '5988', '5826', '3008', '4922', '5081', '1', '37', '4103', '5780', '2217', '6060', '6136', '6709', '6654', '5204', '6114', '3286', '585', '777', '700', '3763', '4808', '2413', '2242', '3811', '3109', '6942', '5476', '4158', '6914', '1882', '277', '5341', '5277', '2218', '715', '1421', '5986', '6766', '1489', '6201', '531', '993', '3891', '6279', '6828', '69', '5770', '5889', '2310', '5506', '2204', '4172', '4422', '3083', '2284', '3437', '21', '4273', '4535', '2546', '6421', '1872', '783', '6138', '753', '4090', '5692', '4031', '2146', '4869', '6838', '5186', '5973', '1201', '4023', '6065', '1976', '231', '6267', '381', '4516', '7083', '735', '5888', '3265', '6332', '5340', '2723', '589', '2508', '6632', '3244', '3250', '3025', '371', '6832', '3638', '6529', '1679', '2971', '2040', '7061', '5146', '1557', '2228', '6409', '6042', '3987', '4409', '5428', '250', '1528', '1888', '4017', '3943', '154', '5507', '2379', '467', '3404', '6765', '7067', '5793', '975', '3006', '5118', '3048', '6687', '6313', '4783', '1412', '6585', '1739', '3350', '5975', '4948', '5914', '2920', '4064', '5762', '1904', '6533', '2680', '3105', '3784', '72', '3997', '4601', '6647', '1708', '1355', '6669', '5742', '1921', '2912', '2161', '6325', '6110', '6371', '5774', '6866', '1107', '4736', '476', '3338', '389', '3935', '817', '2518', '2378', '2826', '5735', '702', '2439', '6271', '6465', '5351', '6616', '5470', '5632', '3275', '6820', '2940', '1366', '2752', '1695', '5858', '5861', '6214', '6008', '6284', '5895', '671', '3440', '2334', '2295', '5908', '1544', '3307', '3858', '6143', '2206', '7088', '6144', '4906', '7140', '1529', '3770', '1165', '3725', '2187', '929', '5797', '3259', '1259', '259', '2396', '96', '6489', '6471', '6241', '3492', '1770', '5264', '4506', '6001', '2050', '2596', '1855', '6040', '1236', '4454', '4239', '1771', '1967', '2178', '1738', '1095', '6342', '6696', '1116', '462', '4462', '3677', '4476', '3186', '6840', '942', '4744', '1238', '2372', '6744', '4641', '3379', '4034', '1643', '3702', '2463', '5423', '6312', '5532', '4655', '6077', '6156', '3771', '5529', '2332', '6606', '3150', '734', '2871', '188', '5607', '6165', '3498', '3046', '1086', '627', '1418', '3529', '4293', '6120', '6721', '4565', '5379', '6813', '5050', '6321', '2907', '4114', '6495', '29', '3757', '2357', '4536', '3384', '3727', '912', '5252', '4138', '6142', '4730', '3037', '6859', '6864', '2691', '3883', '287', '1289', '5985', '3314', '4376', '4647', '6808', '4066', '2028', '2030', '2144', '7026', '2951', '5905', '1047', '1455', '1946', '4157', '1482', '2487', '3261', '2930', '5259', '6012', '3018', '4241', '2200', '794', '184', '2532', '6924', '1409', '4589', '908', '7109', '4287', '5810', '4873', '1342', '1802', '5874', '2857', '4824', '7127', '1703', '3439', '6477', '3780', '5925', '2016', '3582', '2892', '5024', '3537', '4787', '6088', '5612', '1776', '2792', '1881', '2271', '6466', '4677', '2542', '1700', '4921', '4349', '1554', '6863', '2917', '4559', '6664', '2354', '4840', '39', '5304', '4848', '1038', '2466', '1495', '3978', '4726', '1783', '3968', '359', '3173', '4562', '2008', '832', '1381', '2142', '6975', '6535', '6681', '4426', '3900', '3589', '5835', '5803', '1123', '3134', '3486', '3107', '2109', '4085', '4988', '2327', '5007', '318', '4642', '7005', '6662', '3975', '6848', '4169', '5715', '4827', '4032', '4403', '6192', '5000', '3159', '1368', '6353', '6100', '2113', '2174', '7105', '6528', '2522', '6004', '5138', '5820', '6051', '4817', '6461', '1994', '3169', '3838', '859', '6674', '6817', '4024', '6317', '2594', '5838', '4639', '6802', '4133']


good depo = []